# Export data to Supabase database

In [ ]:
%pip install python-dotenv sqlalchemy psycopg2

## NYT

In [ ]:
import pandas as pd
import os
import ast
from dotenv import load_dotenv
import json
from sqlalchemy import create_engine

In [24]:
# Load variables from .env into environment
load_dotenv()

# Retrieve values
user = os.getenv("user")
password = os.getenv("password")
host = os.getenv("host")
port = os.getenv("port")
dbname = os.getenv("dbname")

In [3]:
nyt_data = pd.read_csv("../input-data/nyt-temp-data.csv")

In [5]:
print(type(nyt_data["keywords"].iloc[0]))

<class 'str'>


In [7]:
def parse_python_style_json(x):
    if isinstance(x, str):
        x = x.strip()
        # Try to safely evaluate string that looks like a Python object
        if x.startswith("[") or x.startswith("{"):
            try:
                result = ast.literal_eval(x)
                if isinstance(result, (list, dict)):
                    return result
            except Exception:
                pass  # Return original if it can't be parsed
    return x

In [29]:
json_columns = ["keywords", "multimedia", "byline", "headline"]

for col in json_columns:
    nyt_data[col] = nyt_data[col].apply(json.dumps)

In [10]:
print(type(nyt_data["keywords"].iloc[0])) 

<class 'list'>


In [16]:
nyt_data = nyt_data.drop(labels = "Unnamed: 0", axis=1)

In [27]:
nyt_data

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,headline,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,subsection_name
0,Wrestling with age and a case of idea theft.,https://www.nytimes.com/2024/09/01/business/he...,Wrestling with age and a case of idea theft.,"Send questions about the office, money, career...",BU,3.0,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Help! I’m ‘Older’ and on the Job Hun...,"[{'name': 'subject', 'value': 'Careers and Pro...",2024-09-01T04:01:07+0000,article,SundayBusiness,Business Day,"{'original': 'By Anna Holmes', 'person': [{'fi...",News,nyt://article/da8532bd-f9bd-5ca3-9e7e-afef6e9f...,1280,nyt://article/da8532bd-f9bd-5ca3-9e7e-afef6e9f...,NaN
1,"Grueling shifts, abuse from the public and sub...",https://www.nytimes.com/2024/09/01/world/asia/...,"Grueling shifts, abuse from the public and sub...",Exhausted doctors resting in crowded on-call r...,A,4.0,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'Worked to the Bone, India’s Doctors ...","[{'name': 'glocations', 'value': 'India', 'ran...",2024-09-01T04:01:25+0000,article,Foreign,World,{'original': 'By Anupreeta Das and Pragati K.B...,News,nyt://article/aeabc262-aeb0-5423-a7ac-8bb664cb...,1310,nyt://article/aeabc262-aeb0-5423-a7ac-8bb664cb...,Asia Pacific
2,"About a quarter of the residents of Maracaibo,...",https://www.nytimes.com/2024/09/01/world/ameri...,"About a quarter of the residents of Maracaibo,...",It was once a thriving metropolis in the heart...,A,1.0,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'What Happens When Half a Million Peo...,"[{'name': 'glocations', 'value': 'Maracaibo (V...",2024-09-01T04:01:27+0000,article,Foreign,World,{'original': 'By Frances Robles and Marian Car...,News,nyt://article/42c0d0f2-ea62-5d2b-8eba-baa04180...,1460,nyt://article/42c0d0f2-ea62-5d2b-8eba-baa04180...,Americas
3,Israel and Hamas agreed to pause the war to pe...,https://www.nytimes.com/2024/09/01/world/middl...,Israel and Hamas agreed to pause the war to pe...,Health workers on Sunday began a polio vaccina...,A,1.0,The New York Times,[],"{'main': 'In Race Against Polio, Gaza Begins V...","[{'name': 'subject', 'value': 'Israel-Gaza War...",2024-09-01T04:01:43+0000,article,Foreign,World,"{'original': 'By Bilal Shbair, Erika Solomon a...",News,nyt://article/6393c6c3-0e1f-5494-925d-165e7aaf...,1473,nyt://article/6393c6c3-0e1f-5494-925d-165e7aaf...,Middle East
4,The prospect of a strong performance by the fa...,https://www.nytimes.com/2024/09/01/world/europ...,The prospect of a strong performance by the fa...,"On Sunday, voters in the eastern German states...",NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'What to Watch For in East German Sta...,"[{'name': 'glocations', 'value': 'Germany', 'r...",2024-09-01T04:01:43+0000,article,Foreign,World,"{'original': 'By Christopher F. Schuetze', 'pe...",News,nyt://article/fe046102-78e5-530d-89e0-59ff09c0...,812,nyt://article/fe046102-78e5-530d-89e0-59ff09c0...,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49035,The first Black officer to achieve the rank of...,https://www.nytimes.com/2024/08/31/us/arthur-j...,The first Black officer to achieve the rank of...,"Arthur J. Gregg, the first African American Ar...",NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'Arthur J. Gregg, Trailblazing Army O...","[{'name': 'subject', 'value': 'Deaths (Obituar...",2024-08-31T14:57:12+0000,article,Obits,U.S.,"{'original': 'By Trip Gabriel', 'person': [{'f...",Obituary (Obit),nyt://article/15ef03c9-295b-50e0-a0f4-64f9a182...,1019,nyt://article/15ef03c9-295b-50e0-a0f4-64f9a182...,NaN
49036,"In a reversal, two senior U.S. military offici...",https://www.nytimes.com/2024/08/31/world/europ...,"In a reversal, two senio

In [25]:
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}")

In [30]:
nyt_data.to_sql("nyt_articles", engine, if_exists="replace", index=False)

40

## Zeit